# Zadanie 2

## Treść

Cena europejskiej opcji kupna o cenie wykonania 60 wygasającej za 6 miesięcy wynosi 4. Cena akcji, na którą została wystawiona opcja, to 58. Na akcję wypłacana
będzie dywidenda w wysokości 1 za 2 miesiące i w tej samej wysokości za 5 miesięcy.
Stopa procentowa dla wszystkich okresów to 10% (kapitalizacja ciągła). Ile wynosi
cena europejskiej opcji sprzedaży o cenie wykonania 60 wygasającej za 6 miesięcy?

## Rozwiązanie

Korzystamy ze wzoru:
$$ C - P = S(0)-D_0-e^{-rT}X $$

U nas :
$$
C = 4 \\
X = 60 \\ 
r = 0.1 \\
$$

In [14]:
from math import exp

C = 4
X = 60
D = 1
r = 0.1
T = 0.5 #(1/2 year)
S0 = 58

D0 = exp(-r * 2/12) + exp(-r * 5/12)

part3 = exp(-r*T) * X

# Więc ze wzoru P = C - S(0) + D0 + Exp(X)
P = C + D0 + part3 - S0

print(f"Powinna kosztować {P:.2f}")


Powinna kosztować 5.02


# Zadanie 3

## Treść

Napisz funkcję (oznaczenia jak na wykładzie)
BlackScholesGreeks(S, X, t, r, sigma),

która zwraca wektor
c(CE,PE,DeltaCall, DeltaPut, Gamma, Vega) ,
gdzie

• CE = cena europejskiej opcji kupna wg wzoru Blacka-Scholesa,

• PE = cena europejskiej opcji sprzedaży wg wzoru Blacka-Scholesa,

• DeltaCall = współczynnik grecki delta dla europejskiej opcji kupna,

• DeltaPut = współczynnik grecki delta dla europejskiej opcji sprzedaży,

• Gamma = współczynnik grecki gamma dla opcji europejskiej,

• Vega = współczynnik grecki vega dla opcji europejskiej

Załóżmy, że dany jest portfel V złożony z x akcji S, y obligacji B oraz z
instrumentów pochodnych D na instrument bazowy S:
V (S) = xS + yB + zD(S).

Standardowo wykorzystuje się tu współczynniki greckie:

$$
\delta = \frac{\partial V}{\partial S}, 
\qquad
\gamma = \frac{\partial^2 V}{\partial S^2},
\qquad
\theta = \frac{\partial V}{\partial t}
$$

$$
\text{vega} = \frac{\partial V}{\partial \sigma},
\qquad
\rho = \frac{\partial V}{\partial r}
$$

Tutaj portfel składa się właściwie z jednej opcji

Więc dlatego są dwie delty, bo portfel może mieć wartość call lub put

In [ ]:
import math
from scipy.stats import norm

def BlackScholesGreeks(S, X, t, r, sigma):

    sqrt_t = math.sqrt(t)

    d1 = (math.log(S / X) + (r + 0.5 * sigma**2) * t) / (sigma * sqrt_t)

    d2 = d1 - sigma * sqrt_t

    disc = math.exp(-r * t)

    CE = S * norm.cdf(d1) - X * disc * norm.cdf(d2)

    PE = X * disc * norm.cdf(-d2) - S * norm.cdf(-d1)


    # C - P = S - X * exp(-r * t)

    # Pochodna C po S
    DeltaCall = norm.cdf(d1)
    
    # Pochodna P po S
    DeltaPut  = norm.cdf(d1) - 1.0

    # delta put lub delta call po s
    Gamma = norm.pdf(d1) / (S * sigma * sqrt_t)
    
    # delta put lub delta call po sigma
    Vega  = S * norm.pdf(d1) * sqrt_t

    return CE, PE, DeltaCall, DeltaPut, Gamma, Vega

print(BlackScholesGreeks(80, 100, 2, 0.1, 0.05))
    

(np.float64(1.4672941958825376), np.float64(3.3403695036807193), np.float64(0.3851647578094882), np.float64(-0.6148352421905118), np.float64(0.06758142541274488), np.float64(43.25211226415673))


# Zadanie 4

## Treść

Napisz funkcję (oznaczenia jak na wykładzie)
CRRPrice(TypeFlag, S, X, d, r, u, n),

## Rozwiązanie zadania

$$
1+u=\exp(\sqrt{t}\,\sigma), 
\qquad
1+d=\exp(-\sqrt{t}\,\sigma),
\qquad
1+r=\exp(r_f\,T).
$$

Stąd prawdopodobieństwo neutralne ze względu na ryzyko wynosi

$$
p_*=\frac{r-d}{u-d}
=\frac{\exp(r_f\,T)-\exp(-\sqrt{t}\,\sigma)}{\exp(\sqrt{t}\,\sigma)-\exp(-\sqrt{t}\,\sigma)}.
$$

Jeśli teraz dana jest opcja:

  - o funkcji wypłaty $f(S)$,
  - wygasająca i wykonywana w momencie $T=n\tau$,
  - a samo $n$ jest wystarczająco dużą liczbą naturalną,

to obliczoną w modelu dwumianowym cenę tej opcji dla $k=0$ traktuje się
jako przybliżenie ceny opcji w momencie $t=0$ dla wyjściowego procesu ceny $S(t)$.


$$
C^E(0)=S(0)\Big[1-\Phi(m-1,n,q)\Big]-X(1+r)^{-n}\Big[1-\Phi(m-1,n,p_*)\Big].
$$

$$
P^E(0)=-S(0)\,\Phi(m-1,n,q)+X(1+r)^{-n}\,\Phi(m-1,n,p_*).
$$

$$
\Phi(m,n,p):=\sum_{k=0}^{m}\binom{n}{k}p^{k}(1-p)^{\,n-k}.
$$

m
$$
m_{\text{call}}
=\min\Big\{k\in\{0,1,\dots,n\}:\ S_n(k)\ge X\Big\}.
$$



In [25]:
import math

def Phi(m, n, p):


    s = 0.0
    for k in range(m + 1):
        s += math.comb(n, k) * (p ** k) * ((1.0 - p) ** (n - k))
    return s


def m_call_min(S, X, Up, Down, n):

    for k in range(n + 1):
        Sn = S * (Up ** k) * (Down ** (n - k))
        if Sn >= X:
            return k
        
    return n + 1  



def CRRPrice(TypeFlag, S, X, u, d, r, n):

    Up = 1.0 + u
    Down = 1.0 + d
    R = 1.0 + r

    p = (r - d) / (u - d)
    q = (p * Up) / R

    m = m_call_min(S, X, Up, Down, n)

    Phi_m1_q = Phi(m - 1, n, q)
    Phi_m1_p = Phi(m - 1, n, p)

    disc = R ** (-n)

    if TypeFlag == "ce":
        return S * (1.0 - Phi_m1_q) - X * disc * (1.0 - Phi_m1_p)
    else:  
        return -S * Phi_m1_q + X * disc * Phi_m1_p


print(CRRPrice("pe", 80, 80, 0.1, -0.05, 0.05, 2))


0.7860922146636478
